# User interface

Load the sets and parameters from file.

In [1]:
import json
import ipyaggrid as pg
import mola.specification4a as ms
model_parameters_file_name = 'model_parameters_data.json'
model_sets_file_name = 'model_set_data.json'
with open(model_parameters_file_name) as fp:
    params_json = json.load(fp)
with open(model_sets_file_name) as fp:
    sets_json = json.load(fp)
sets_json

{'F_m': ['1f7bbd3e-fcd1-412d-8608-035b855ea735'],
 'F_s': [],
 'F_t': ['0ace02fa-eca5-482d-a829-c18e46a52db4',
  '4ecf1190-7028-4038-89b0-79adeb3e98cc',
  '559d5695-102a-4beb-9f7f-3db334c9b51d'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
  'cfc3f58f-7bdf-3019-888b-c4f2dbc332e8',
  '760f1858-b0ef-3280-930e-88ebe4645061'],
 'P_t': ['44ad59ca-4fe0-394c-a6d9-5dea68783c23',
  'f615ae37-fa6d-4a47-958c-23234c986400',
  '9106bf8d-869d-425f-b8ee-0bf9a78bcf65',
  '97aa2533-ae98-3f8f-b3e2-3acb9d66a010',
  'b37252a8-fbb5-47cb-a3aa-1485d090bdfd',
  '660ba3fe-d7a5-4c43-8777-27d331934edf',
  'fe7d1840-4341-40fd-ba6d-fbba126ca7d3'],
 'P_s': [],
 'KPI': ['061b7db5-4f56-3368-bf50-9ff0fcc8dd1f'],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

Get the structure of the parameters from a specification object.

In [2]:
spec = ms.ScheduleSpecification()
spec.user_defined_parameters

{'C': {'index': ['F_m', 'K', 'D', 'T'],
  'doc': 'Conversion factor for material flows'},
 'Demand': {'index': ['D', 'K', 'T'], 'doc': 'Specific demand'},
 'Total_Demand': {'index': ['D', 'K'], 'doc': 'Total demand'},
 'L': {'index': ['F_m', 'F_t'],
  'doc': 'Binary conversion factor between material and transport flows'},
 'X': {'index': ['K', 'T'], 'doc': 'Latitude'},
 'Y': {'index': ['K', 'T'], 'doc': 'Longitude'},
 'd': {'index': ['P', 'F_m', 'K', 'T'], 'doc': 'Distance'},
 'phi': {'index': ['F', 'P', 'T'], 'doc': 'Cost impact factor'},
 'J': {'index': ['F_m', 'P_m', 'F_t', 'P_t'],
  'doc': 'Binary conversion factor between material and transport flows'}}

Get the lookup tables for sets.

In [3]:
import mola.dataview as dv
import mola.dataimport as di
import pandas as pd
conn = di.get_sqlite_connection()
lookup = dv.get_lookup_tables(conn, single_column=True)

SELECT "REF_ID","NAME" FROM "TBL_CATEGORIES"
SELECT "REF_ID" "FLOW_REF_ID","NAME" FROM "TBL_FLOWS"
SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION_NAME" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID"
SELECT "REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW'
SELECT "TBL_IMPACT_METHODS"."NAME" "method_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "category_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID"


Use an aggrid widget to show parameters with text replacing reference ids.

In [4]:
import ipywidgets as widgets
from IPython.display import clear_output
current_param = 'J'
dropdown = widgets.Dropdown(
    options=params_json.keys(),
    description='Parameter:',
    value=current_param
)
def change_parameter(change):
    clear_output(wait=True)
    display(dropdown)
    current_param = change.new
    df = pd.DataFrame(params_json[current_param])
    index_sets = spec.user_defined_parameters[current_param]['index']
    df[index_sets] = pd.DataFrame(df["index"].to_list(), columns=index_sets)
    df = df.drop('index', axis=1)
    for k in index_sets:
        if k in lookup.keys():
            df[k] = df[k].map(lookup[k][k])
    g = pg.Grid(grid_data=df, grid_options={'columnDefs' : [{'field': c} for c in df.columns]}, show_toggle_edit=True)
    display(g)
    
dropdown.observe(change_parameter, names='value')
display(dropdown)
df = pd.DataFrame(params_json[current_param])
index_sets = spec.user_defined_parameters[current_param]['index']
df[index_sets] = pd.DataFrame(df["index"].to_list(), columns=index_sets)
df = df.drop('index', axis=1)
for k in index_sets:
    if k in lookup.keys():
        df[k] = df[k].map(lookup[k][k])
g = pg.Grid(grid_data=df, grid_options={'columnDefs' : [{'field': c} for c in df.columns]}, show_toggle_edit=True)
display(g)

Dropdown(description='Parameter:', index=1, options=('C', 'J', 'Total_Demand', 'd', 'phi'), value='J')

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': […

The binary link factor $J$ can have many different values because it links material processes to transport processes. It is hard to see what is linked with a table.

In [5]:
from pyvis.network import Network
import networkx as ns
df = pd.DataFrame(params_json['J'])
df[index_sets] = pd.DataFrame(df["index"].to_list(), columns=index_sets)
df = df.drop('index', axis=1)
df

,value,F_m,P_m,F_t,P_t
0,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,0ace02fa-eca5-482d-a829-c18e46a52db4,44ad59ca-4fe0-394c-a6d9-5dea68783c23
1,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,0ace02fa-eca5-482d-a829-c18e46a52db4,f615ae37-fa6d-4a47-958c-23234c986400
2,1.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,0ace02fa-eca5-482d-a829-c18e46a52db4,9106bf8d-869d-425f-b8ee-0bf9a78bcf65
3,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,0ace02fa-eca5-482d-a829-c18e46a52db4,97aa2533-ae98-3f8f-b3e2-3acb9d66a010
4,1.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,0ace02fa-eca5-482d-a829-c18e46a52db4,b37252a8-fbb5-47cb-a3aa-1485d090bdfd
...,...,...,...,...,...
58,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,760f1858-b0ef-3280-930e-88ebe4645061,559d5695-102a-4beb-9f7f-3db334c9b51d,9106bf8d-869d-425f-b8ee-0bf9a78bcf65
59,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,760f1858-b0ef-3280-930e-88ebe4645061,559d5695-102a-4beb-9f7f-3db334c9b51d,97aa2533-ae98-3f8f-b3e2-3acb9d66a010
60,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,760f1858-b0ef-3280-930e-88ebe4645061,559d5695-102a-4beb-9f7f-3db334c9b51d,b37252a8-fbb5-47cb-a3aa-1485d090bdfd
61,0.0,1f7bbd3e-fcd1-412d-8608-035b855ea735,760f1858-b0ef-3280-930e-88ebe4645061,559d5695-102a-4beb-9f7f-3db334c9b51d,660ba3fe-d7a5-4c43-8777-27d331934edf


We show a graphical representation of the network with nodes representing processes.

In [6]:
g = Network(width='100%', height=1000, heading='Material Transport Network', notebook=True)
#g.show_buttons()
nodes = sets_json['P_m'] + sets_json['P_t']
titles = [lookup['P']['P'][n] for n in nodes]
titles = [t.replace('|', '\n') for t in titles]
colors = ['#FF9999'  if n in sets_json['P_m'] else '#9999FF' for n in nodes]
levels = [0  if n in sets_json['P_m'] else 1 for n in nodes]
nodes += ['output']
titles += ['output']
colors += ['black']
levels += [2]
#g.add_nodes(nodes, title = titles, color=colors, level=levels)
for i in range(len(nodes)):
    g.add_node(nodes[i], title=nodes[i], label=titles[i], color=colors[i], level=levels[i], mass=2)
g.show("nodes.html")

Then define the links using each row of the table.

In [7]:
edges = []
for index, row in df.iterrows():
    if row['value']:
        g.add_edge(row['P_m'], row['P_t'], title=row['F_m'])
        g.add_edge(row['P_t'], 'output', title=row['F_t'])

g.set_options("""
var options = {
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "layout": {
    "hierarchical": {
      "enabled": true,
      "direction": "LR",
      "levelSeparation": 300
    }
  },
  "physics": {
    "enabled": false,
    "hierarchicalRepulsion": {
      "centralGravity": 0
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  }
}
""")

The `pyvis` package is used to draw the network. It writes out an HTML file with Javascript to the file system.

In [8]:
g.show("material_transport_network.html")